In [ ]:
import os

input_file = r"C:\Users\didi\proposal skripsi\GitarKlasik.m4a"
if os.path.exists(input_file):
    print("File exists!")
else:
    print("File not found!")

In [ ]:
import os, sys
from os import path
from pydub import AudioSegment
from pathlib import Path

AudioSegment.from_file("GitarKlasik.m4a", format='m4a').export("GitarKlasikWav.wav", format="wav")

In [16]:
from pydub import AudioSegment
import math

class SplitWavAudio():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '\\' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)

    def rms_normalize(chunk, target_dBFS=-20):
        """Normalisasi berdasarkan RMS agar loudness lebih seragam."""
        change_in_dBFS = target_dBFS - chunk.dBFS
        return chunk.apply_gain(change_in_dBFS)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split_per_min(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '\\' + split_filename, format="wav")

    def single_split_and_normalize_per_sec(self, from_min, to_min, split_filename):
        t1 = from_min * 1000
        t2 = to_min * 1000
        normalized_audio = self.rms_normalize(self.audio[t1:t2])
        normalized_audio.export(self.folder + '\\' + split_filename, format="wav")
        
    def multiple_split(self, sec_per_split):
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, sec_per_split):
            split_fn = str(int(i/sec_per_split)) + '_' + self.filename
            self.single_split(i, i+sec_per_split, split_fn)
            print(str(int(i/sec_per_split)) + ' Done')
            if i == total_secs - sec_per_split:
                print('All splited successfully')

    def multiple_split_and_normalize_per_sec(self, sec_per_split):
        total_secs = int(self.get_duration())
        for i in range(0, total_secs, sec_per_split):
            split_fn = str(int(i/sec_per_split)) + '_' + self.filename
            self.single_split_and_normalize_per_sec(i, i+sec_per_split, split_fn)
            print(str(int(i/sec_per_split)) + ' Done')
            if i == total_secs - sec_per_split:
                print('All splited successfully')

def rms_normalize(chunk, target_dBFS=-20):
        """Normalisasi berdasarkan RMS agar loudness lebih seragam."""
        change_in_dBFS = target_dBFS - chunk.dBFS
        return chunk.apply_gain(change_in_dBFS)

def split_audio_by_seconds(folder, file_name, chunk_length_sec):
    # Load the audio file
    file_path = folder + "\\" + file_name
    audio = AudioSegment.from_file(file_path)

    # Convert chunk length from seconds to milliseconds
    chunk_length_ms = chunk_length_sec * 1000

    # Initialize list to store chunks
    chunks = []

    # Loop through and split the audio
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        normalized_chunk = rms_normalize(chunk)
        chunks.append(normalized_chunk)

        # Optionally export each chunk to separate files
        split_fn = str(int(i/chunk_length_ms)) + '_' + file_name
        chunk.export(folder + '\\' + split_fn, format="wav")

    return chunks


In [ ]:
folder = "C:\\Users\\didi\\proposal skripsi"
file = 'GitarKlasikWav.wav'
split_wav = SplitWavAudio(folder, file)
split_wav.single_split(from_min = 0, to_min = 30, split_filename = "GitarKlasikFirstThirtyMin.wav")

In [5]:
from pydub import AudioSegment

# Load your audio file
audio = AudioSegment.from_file("C:/Users/didi/proposal skripsi/GitarKlasikFull/GitarKlasikWav.wav")

# Get the number of channels
channels = audio.channels
print(f"Number of Channels: {channels}")

Number of Channels: 2


In [ ]:
from pydub import AudioSegment

# Load your stereo audio file
stereo_audio = AudioSegment.from_file("GitarKlasik.wav")

# Convert stereo to mono by averaging both channels
mono_audio = stereo_audio.set_channels(1)

# Export the mono audio
mono_audio.export("mono_GitarKlasik.wav", format="wav", )

print("Stereo audio successfully converted to mono!")

In [ ]:
path ='C:\\Users\\didi\\proposal skripsi\\mono_GitarKlasik.wav'
rate, audio = read(path)

In [73]:
from pydub import AudioSegment

# Load file audio
audio = AudioSegment.from_file("mono_GitarKlasik.wav")

# Resample ke 16 kHz
audio_16kHz = audio.set_frame_rate(16000)

# Simpan file hasil resampling
audio_16kHz.export("resampled_GitarKlasik.wav", format="wav")

print("Audio berhasil di-resample ke 16 kHz!")

Audio berhasil di-resample ke 16 kHz!


In [75]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import split_on_silence

# Load audio file
audio = AudioSegment.from_file("resampled_GitarKlasik.wav")

# Normalisasi ringan sebelum pemotongan (gunakan headroom untuk menghindari clipping)
normalized_audio = normalize(audio, headroom=5.0)  # Headroom 5 dB untuk normalisasi ringan

# Simpan hasil normalisasi sebelum pemotongan (opsional)
normalized_audio.export("normalized_GitarKlasik.wav", format="wav")

<_io.BufferedRandom name='normalized_GitarKlasik.wav'>

In [77]:
# Import required libraries
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write
import numpy as np
# Pass audio path
path ='C:\\Users\\didi\\proposal skripsi\\normalized_GitarKlasik.wav'
rate, audio = read(path)
# make the audio in pydub audio segment format
aud = AudioSegment.from_file("normalized_GitarKlasik.wav")
print(rate)
# use split on sience method to split the audio based on the silence, 
# here we can pass the min_silence_len as silent length threshold in ms and intensity thershold
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 400,
    silence_thresh = -50,
    keep_silence = 200,)
#audio chunks are combined here
audio_processed = sum(audio_chunks)
audio_processed.export('C:\\Users\\didi\\proposal skripsi\\SilenceRemoved_GitarKlasik.wav', format = "wav")
audio_processed = np.array(audio_processed.get_array_of_samples())
#Note the processed audio rate is not the same - it would be 1K
path ='C:\\Users\\didi\\proposal skripsi\\SilenceRemoved_GitarKlasik.wav'
rate, audio = read(path)
print(rate)

16000
16000


In [147]:
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\GitarKlasik1Detik", "GitarKlasik.wav", chunk_length_sec = 1)

In [149]:
GitarKlasik3DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\GitarKlasik3Detik", "GitarKlasik.wav", chunk_length_sec = 3)

In [151]:
GitarKlasik5DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\GitarKlasik5Detik", "GitarKlasik.wav", chunk_length_sec = 5)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(358):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/GitarKlasik5Detik/{i}_GitarKlasik.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/GitarKlasik5Detik/gambarGelombangGitarKlasik5Detik/{i}_GitarKlasik.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/0_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/1_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/2_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/3_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/4_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/5_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/6_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/7_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/8_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik5Detik/9_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skri

In [7]:
filename = f"C:/Users/didi/proposal skripsi/GitarKlasik5Detik/358_GitarKlasik.wav"  # Format nama file

sampling_rate, data = wavfile.read(filename)

duration = len(data) / sampling_rate
time = np.linspace(0, duration, num=len(data))

plt.figure(figsize=(10, 4))
plt.plot(time, data, label="Waveform")
plt.xlabel("Waktu (detik)")
plt.ylabel("Amplitudo")
plt.title(f"Gelombang Suara - {filename}")
plt.legend()

plt.savefig(f"C:/Users/didi/proposal skripsi/GitarKlasik5Detik/gambarGelombangGitarKlasik5Detik/358_GitarKlasik.png", dpi=300)
plt.close()  # Tutup plot agar tidak memakan memori

In [1]:
from pydub import AudioSegment

kecapi = AudioSegment.from_mp3("Midnight Healing Music _ Sitar Instrumental _ Classical Music _ Raga Darbari kanada [dKMVeZEJjBo].mp3")
tiga_puluh_menit = (30 * 60 * 1000) + 5000
first_30_minutes = kecapi[:tiga_puluh_menit]
first_30_minutes.export("sitar_30_min.mp3", 
                        format="mp3") 
print("New Audio file is created and saved") 

New Audio file is created and saved


In [3]:
AudioSegment.from_file("sitar_30_min.mp3", format='mp3').export("sitar_30_min_wav.wav", format="wav")

<_io.BufferedRandom name='sitar_30_min_wav.wav'>

In [7]:
from pydub import AudioSegment

# Load your stereo audio file
stereo_audio = AudioSegment.from_file("sitar_30_min_wav.wav")

# Convert stereo to mono by averaging both channels
mono_audio = stereo_audio.set_channels(1)

# Export the mono audio
mono_audio.export("mono_sitar_30_min_wav.wav", format="wav", )

print("Stereo audio successfully converted to mono!")

Stereo audio successfully converted to mono!


In [9]:
from pydub import AudioSegment

# Load file audio
audio = AudioSegment.from_file("mono_sitar_30_min_wav.wav")

# Resample ke 16 kHz
audio_16kHz = audio.set_frame_rate(16000)

# Simpan file hasil resampling
audio_16kHz.export("resampled_mono_sitar_30_min_wav.wav", format="wav")

print("Audio berhasil di-resample ke 16 kHz!")

Audio berhasil di-resample ke 16 kHz!


In [11]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import split_on_silence

# Load audio file
audio = AudioSegment.from_file("resampled_mono_sitar_30_min_wav.wav")

# Normalisasi ringan sebelum pemotongan (gunakan headroom untuk menghindari clipping)
normalized_audio = normalize(audio, headroom=5.0)  # Headroom 5 dB untuk normalisasi ringan

# Simpan hasil normalisasi sebelum pemotongan (opsional)
normalized_audio.export("normalized_resampled_mono_sitar_30_min_wav.wav", format="wav")

print("normalisasi selesai")

normalisasi selesai


In [13]:
# Import required libraries
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write
import numpy as np
# Pass audio path
path ='C:\\Users\\didi\\proposal skripsi\\normalized_resampled_mono_sitar_30_min_wav.wav'
rate, audio = read(path)
# make the audio in pydub audio segment format
aud = AudioSegment.from_file("normalized_resampled_mono_sitar_30_min_wav.wav")
print(rate)
# use split on sience method to split the audio based on the silence, 
# here we can pass the min_silence_len as silent length threshold in ms and intensity thershold
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 400,
    silence_thresh = -50,
    keep_silence = 200,)
#audio chunks are combined here
audio_processed = sum(audio_chunks)
audio_processed.export('C:\\Users\\didi\\proposal skripsi\\silenceRemoved_normalized_resampled_mono_sitar_30_min_wav.wav', format = "wav")
audio_processed = np.array(audio_processed.get_array_of_samples())
#Note the processed audio rate is not the same - it would be 1K
path ='C:\\Users\\didi\\proposal skripsi\\silenceRemoved_normalized_resampled_mono_sitar_30_min_wav.wav'
rate, audio = read(path)
print(rate)

16000
16000


In [19]:
Sitar1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\Sitar1Detik", "sitar.wav", chunk_length_sec = 1)

In [21]:
Sitar1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\Sitar3Detik", "sitar.wav", chunk_length_sec = 3)

In [23]:
Sitar5DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\Sitar5Detik", "sitar.wav", chunk_length_sec = 5)

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(597):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/GitarKlasik3Detik/{i}_GitarKlasik.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/GitarKlasik3Detik/gambarGelombangGitarKlasik3Detik/{i}_GitarKlasik.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/0_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/1_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/2_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/3_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/4_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/5_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/6_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/7_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/8_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik3Detik/9_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skri

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(1793):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/GitarKlasik1Detik/{i}_GitarKlasik.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/GitarKlasik1Detik/gambarGelombangGitarKlasik1Detik/{i}_GitarKlasik.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/0_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/1_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/2_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/3_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/4_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/5_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/6_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/7_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/8_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/GitarKlasik1Detik/9_GitarKlasik.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skri

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(360):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/Sitar5Detik/{i}_Sitar.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/Sitar5Detik/gambarSitar5Detik/{i}_Sitar.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/0_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/1_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/2_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/3_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/4_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/5_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/6_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/7_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/8_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/9_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/10_Sitar.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/Sitar5Detik/11_Sitar.wav telah 

In [77]:
from pydub import AudioSegment
import os

kecapi1 = AudioSegment.from_mp3("kecapi1.mp3")
kecapi2 = AudioSegment.from_mp3("kecapi2.mp3")
kecapi3 = AudioSegment.from_mp3("kecapi3.mp3")
kecapi4 = AudioSegment.from_mp3("kecapi4.mp3")
kecapi5 = AudioSegment.from_mp3("kecapi5.mp3")
kecapi6 = AudioSegment.from_mp3("kecapi6.mp3")
kecapi7 = AudioSegment.from_mp3("kecapi7.mp3")

kecapi_combined = kecapi1 + kecapi2 +kecapi3 +kecapi4 +kecapi5 +kecapi6 +kecapi7

kecapi_combined.export("kecapi_combined.wav", format="wav", )

<_io.BufferedRandom name='kecapi_combined.wav'>

In [81]:
from pydub import AudioSegment

kecapi = AudioSegment.from_wav("kecapi_combined.wav")
tiga_puluh_menit = (30 * 60 * 1000) + 5000
first_30_minutes = kecapi[:tiga_puluh_menit]
first_30_minutes.export("kecapi_combined_30min.wav", 
                        format="wav") 
print("New Audio file is created and saved") 

New Audio file is created and saved


In [83]:
from pydub import AudioSegment

# Load your stereo audio file
stereo_audio = AudioSegment.from_file("kecapi_combined_30min.wav")

# Convert stereo to mono by averaging both channels
mono_audio = stereo_audio.set_channels(1)

# Export the mono audio
mono_audio.export("kecapi_combined_30min_mono.wav", format="wav", )

print("Stereo audio successfully converted to mono!")

Stereo audio successfully converted to mono!


In [85]:
from pydub import AudioSegment

# Load file audio
audio = AudioSegment.from_file("kecapi_combined_30min_mono.wav")

# Resample ke 16 kHz
audio_16kHz = audio.set_frame_rate(16000)

# Simpan file hasil resampling
audio_16kHz.export("kecapi_combined_30min_mono_resampled.wav", format="wav")

print("Audio berhasil di-resample ke 16 kHz!")

Audio berhasil di-resample ke 16 kHz!


In [87]:
# Import required libraries
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write
import numpy as np
# Pass audio path
path ='C:\\Users\\didi\\proposal skripsi\\kecapi_combined_30min_mono_resampled.wav'
rate, audio = read(path)
# make the audio in pydub audio segment format
aud = AudioSegment.from_file("kecapi_combined_30min_mono_resampled.wav")
print(rate)
# use split on sience method to split the audio based on the silence, 
# here we can pass the min_silence_len as silent length threshold in ms and intensity thershold
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 400,
    silence_thresh = -50,
    keep_silence = 200,)
#audio chunks are combined here
audio_processed = sum(audio_chunks)
audio_processed.export('C:\\Users\\didi\\proposal skripsi\\kecapi_combined_30min_mono_resampled_silenceRemoved.wav', format = "wav")
audio_processed = np.array(audio_processed.get_array_of_samples())
#Note the processed audio rate is not the same - it would be 1K
path ='C:\\Users\\didi\\proposal skripsi\\kecapi_combined_30min_mono_resampled_silenceRemoved.wav'
rate, audio = read(path)
print(rate)

16000
16000


In [91]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import split_on_silence

# Load audio file
audio = AudioSegment.from_file("kecapi_combined_30min_mono_resampled_silenceRemoved.wav")

# Normalisasi ringan sebelum pemotongan (gunakan headroom untuk menghindari clipping)
normalized_audio = normalize(audio, headroom=5.0)  # Headroom 5 dB untuk normalisasi ringan

# Simpan hasil normalisasi sebelum pemotongan (opsional)
normalized_audio.export("kecapi_combined_30min_mono_resampled_silenceRemoved_normalized.wav", format="wav")

print("normalisasi selesai")

normalisasi selesai


In [93]:
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\kecapi1Detik", "kecapi.wav", chunk_length_sec = 1)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\kecapi3Detik", "kecapi.wav", chunk_length_sec = 3)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\kecapi5Detik", "kecapi.wav", chunk_length_sec = 5)

In [2]:
from pydub import AudioSegment

kecapi = AudioSegment.from_mp3("1 Hour of Relaxing Ukulele Fingerstyle Background Music [pbj1kIaIfdg](udah dicek 30 menit pertamanya, aman.mp3")
tiga_puluh_menit = (30 * 60 * 1000) + 5000
first_30_minutes = kecapi[:tiga_puluh_menit]
first_30_minutes.export("ukulele_30min.wav", 
                        format="wav") 
print("New Audio file is created and saved") 

New Audio file is created and saved


In [4]:
from pydub import AudioSegment

# Load your stereo audio file
stereo_audio = AudioSegment.from_file("ukulele_30min.wav")

# Convert stereo to mono by averaging both channels
mono_audio = stereo_audio.set_channels(1)

# Export the mono audio
mono_audio.export("ukulele_30min_mono.wav", format="wav", )

print("Stereo audio successfully converted to mono!")

Stereo audio successfully converted to mono!


In [8]:
from pydub import AudioSegment

# Load file audio
audio = AudioSegment.from_file("ukulele_30min_mono.wav")

# Resample ke 16 kHz
audio_16kHz = audio.set_frame_rate(16000)

# Simpan file hasil resampling
audio_16kHz.export("ukulele_30min_mono_resampled.wav", format="wav")

print("Audio berhasil di-resample ke 16 kHz!")

Audio berhasil di-resample ke 16 kHz!


In [10]:
# Import required libraries
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write
import numpy as np
# Pass audio path
path ='C:\\Users\\didi\\proposal skripsi\\ukulele_30min_mono_resampled.wav'
rate, audio = read(path)
# make the audio in pydub audio segment format
aud = AudioSegment.from_file("ukulele_30min_mono_resampled.wav")
print(rate)
# use split on sience method to split the audio based on the silence, 
# here we can pass the min_silence_len as silent length threshold in ms and intensity thershold
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 400,
    silence_thresh = -50,
    keep_silence = 200,)
#audio chunks are combined here
audio_processed = sum(audio_chunks)
audio_processed.export('C:\\Users\\didi\\proposal skripsi\\ukulele_30min_mono_resampled_silenceRemoved.wav', format = "wav")
audio_processed = np.array(audio_processed.get_array_of_samples())
#Note the processed audio rate is not the same - it would be 1K
path ='C:\\Users\\didi\\proposal skripsi\\ukulele_30min_mono_resampled_silenceRemoved.wav'
rate, audio = read(path)
print(rate)

16000
16000


In [12]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import split_on_silence

# Load audio file
audio = AudioSegment.from_file("ukulele_30min_mono_resampled_silenceRemoved.wav")

# Normalisasi ringan sebelum pemotongan (gunakan headroom untuk menghindari clipping)
normalized_audio = normalize(audio, headroom=5.0)  # Headroom 5 dB untuk normalisasi ringan

# Simpan hasil normalisasi sebelum pemotongan (opsional)
normalized_audio.export("ukulele_30min_mono_resampled_silenceRemoved_normalized.wav", format="wav")

print("normalisasi selesai")

normalisasi selesai


In [18]:
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\ukulele1Detik", "ukulele.wav", chunk_length_sec = 1)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\ukulele3Detik", "ukulele.wav", chunk_length_sec = 3)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\ukulele5Detik", "ukulele.wav", chunk_length_sec = 5)

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(358):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/kecapi5Detik/{i}_kecapi.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/kecapi5Detik/gambarKecapi5Detik/{i}_kecapi.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/0_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/1_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/2_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/3_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/4_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/5_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/6_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/7_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/8_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/9_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5Detik/10_kecapi.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/kecapi5De

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(358):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/ukulele5Detik/{i}_ukulele.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/ukulele5Detik/gambarUkulele5Detik/{i}_ukulele.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/0_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/1_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/2_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/3_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/4_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/5_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/6_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/7_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/8_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/9_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/ukulele5Detik/10_ukulele.wav telah disimpan.
Grafik untuk C:/Users/didi/prop

In [26]:
from pydub import AudioSegment
import os

kecapi1 = AudioSegment.from_mp3("harpa1.mp3")
kecapi2 = AudioSegment.from_mp3("harpa2.mp3")
kecapi3 = AudioSegment.from_mp3("harpa3.mp3")
kecapi4 = AudioSegment.from_mp3("harpa4.mp3")
kecapi5 = AudioSegment.from_mp3("harpa5.mp3")
kecapi6 = AudioSegment.from_mp3("harpa6.mp3")

kecapi_combined = kecapi1 + kecapi2 +kecapi3 +kecapi4 +kecapi5 +kecapi6

kecapi_combined.export("harpa_combined.wav", format="wav", )

<_io.BufferedRandom name='harpa_combined.wav'>

In [28]:
from pydub import AudioSegment

kecapi = AudioSegment.from_mp3("harpa_combined.wav")
tiga_puluh_menit = (30 * 60 * 1000) + 5000
first_30_minutes = kecapi[:tiga_puluh_menit]
first_30_minutes.export("harpa_30min.wav", 
                        format="wav") 
print("New Audio file is created and saved") 

New Audio file is created and saved


In [30]:
from pydub import AudioSegment

# Load file audio
audio = AudioSegment.from_file("harpa_30min.wav")

# Resample ke 16 kHz
audio_16kHz = audio.set_frame_rate(16000)

# Simpan file hasil resampling
audio_16kHz.export("harpa_30min_resampled.wav", format="wav")

print("Audio berhasil di-resample ke 16 kHz!")

Audio berhasil di-resample ke 16 kHz!


In [32]:
from pydub import AudioSegment

# Load your stereo audio file
stereo_audio = AudioSegment.from_file("harpa_30min_resampled.wav")

# Convert stereo to mono by averaging both channels
mono_audio = stereo_audio.set_channels(1)

# Export the mono audio
mono_audio.export("harpa_30min_resampled_mono.wav", format="wav", )

print("Stereo audio successfully converted to mono!")

Stereo audio successfully converted to mono!


In [34]:
# Import required libraries
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write
import numpy as np
# Pass audio path
path ='C:\\Users\\didi\\proposal skripsi\\harpa_30min_resampled_mono.wav'
rate, audio = read(path)
# make the audio in pydub audio segment format
aud = AudioSegment.from_file("harpa_30min_resampled_mono.wav")
print(rate)
# use split on sience method to split the audio based on the silence, 
# here we can pass the min_silence_len as silent length threshold in ms and intensity thershold
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 400,
    silence_thresh = -50,
    keep_silence = 200,)
#audio chunks are combined here
audio_processed = sum(audio_chunks)
audio_processed.export('C:\\Users\\didi\\proposal skripsi\\harpa_30min_resampled_mono_silenceRemoved.wav', format = "wav")
audio_processed = np.array(audio_processed.get_array_of_samples())
#Note the processed audio rate is not the same - it would be 1K
path ='C:\\Users\\didi\\proposal skripsi\\harpa_30min_resampled_mono_silenceRemoved.wav'
rate, audio = read(path)
print(rate)

16000
16000


In [38]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import split_on_silence

# Load audio file
audio = AudioSegment.from_file("harpa_30min_resampled_mono_silenceRemoved.wav")

# Normalisasi ringan sebelum pemotongan (gunakan headroom untuk menghindari clipping)
normalized_audio = normalize(audio, headroom=5.0)  # Headroom 5 dB untuk normalisasi ringan

# Simpan hasil normalisasi sebelum pemotongan (opsional)
normalized_audio.export("harpa_30min_resampled_mono_silenceRemoved_normalized.wav", format="wav")

print("normalisasi selesai")

normalisasi selesai


In [40]:
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\harpa1Detik", "harpa.wav", chunk_length_sec = 1)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\harpa3Detik", "harpa.wav", chunk_length_sec = 3)
GitarKlasik1DetikChunks = split_audio_by_seconds("C:\\Users\\didi\\proposal skripsi\\harpa5Detik", "harpa.wav", chunk_length_sec = 5)

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Loop untuk banyak file
for i in range(319):  # Ganti dengan jumlah file yang kamu punya
    filename = f"C:/Users/didi/proposal skripsi/harpa5Detik/{i}_harpa.wav"  # Format nama file
    
    try:
        # Baca file WAV
        sampling_rate, data = wavfile.read(filename)

        # Buat sumbu waktu
        duration = len(data) / sampling_rate
        time = np.linspace(0, duration, num=len(data))

        # Plot gelombang
        plt.figure(figsize=(10, 4))
        plt.plot(time, data, label="Waveform")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")
        plt.title(f"Gelombang Suara - {filename}")
        plt.legend()

        # Simpan gambar
        plt.savefig(f"C:/Users/didi/proposal skripsi/harpa5Detik/gambarHarpa5Detik/{i}_harpa.png", dpi=300)
        plt.close()  # Tutup plot agar tidak memakan memori

        print(f"Grafik untuk {filename} telah disimpan.")

    except FileNotFoundError:
        print(f"File {filename} tidak ditemukan, lanjut ke file berikutnya.")

print("proses selesai")

Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/0_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/1_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/2_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/3_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/4_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/5_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/6_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/7_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/8_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/9_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/10_harpa.wav telah disimpan.
Grafik untuk C:/Users/didi/proposal skripsi/harpa5Detik/11_harpa.wav telah 

In [46]:
import os
import librosa
import numpy as np
import pandas as pd

# --- Konfigurasi ---
audio_dir = "C:\\Users\\didi\\proposal skripsi\\GitarKlasik5Detik"  # Ganti dengan path folder kamu
sample_rate = 16000
n_mfcc = 13  # Jumlah koefisien MFCC yang diambil
output_csv = "gitar_mfcc_features.csv"  # File hasil

# --- Ekstraksi MFCC ---
def extract_mfcc(file_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)  # Rata-rata per koefisien
    return mfcc_mean

# --- Loop untuk semua file ---
features = []
for i in range(300):  # atau len(os.listdir(audio_dir)) jika fleksibel
    filename = f"{i}_GitarKlasik.wav"
    path = os.path.join(audio_dir, filename)
    
    if os.path.exists(path):
        mfcc = extract_mfcc(path, sr=sample_rate, n_mfcc=n_mfcc)
        features.append([filename] + mfcc.tolist())
    else:
        print(f"File tidak ditemukan: {filename}")

# --- Simpan ke CSV ---
columns = ["filename"] + [f"mfcc_{i+1}" for i in range(n_mfcc)]
df = pd.DataFrame(features, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Selesai! Fitur MFCC disimpan di: {output_csv}")

C:\Users\didi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Selesai! Fitur MFCC disimpan di: gitar_mfcc_features.csv


In [50]:
import os
import librosa
import numpy as np
import pandas as pd

# --- Konfigurasi ---
audio_dir = "C:\\Users\\didi\\proposal skripsi\\harpa5Detik"  # Ganti dengan path folder kamu
sample_rate = 16000
n_mfcc = 13  # Jumlah koefisien MFCC yang diambil
output_csv = "harpa_mfcc_features.csv"  # File hasil

# --- Ekstraksi MFCC ---
def extract_mfcc(file_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)  # Rata-rata per koefisien
    return mfcc_mean

# --- Loop untuk semua file ---
features = []
for i in range(300):  # atau len(os.listdir(audio_dir)) jika fleksibel
    filename = f"{i}_harpa.wav"
    path = os.path.join(audio_dir, filename)
    
    if os.path.exists(path):
        mfcc = extract_mfcc(path, sr=sample_rate, n_mfcc=n_mfcc)
        features.append([filename] + mfcc.tolist())
    else:
        print(f"File tidak ditemukan: {filename}")

# --- Simpan ke CSV ---
columns = ["filename"] + [f"mfcc_{i+1}" for i in range(n_mfcc)]
df = pd.DataFrame(features, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Selesai! Fitur MFCC disimpan di: {output_csv}")

Selesai! Fitur MFCC disimpan di: harpa_mfcc_features.csv


In [52]:
import os
import librosa
import numpy as np
import pandas as pd

# --- Konfigurasi ---
audio_dir = "C:\\Users\\didi\\proposal skripsi\\kecapi5Detik"  # Ganti dengan path folder kamu
sample_rate = 16000
n_mfcc = 13  # Jumlah koefisien MFCC yang diambil
output_csv = "kecapi_mfcc_features.csv"  # File hasil

# --- Ekstraksi MFCC ---
def extract_mfcc(file_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)  # Rata-rata per koefisien
    return mfcc_mean

# --- Loop untuk semua file ---
features = []
for i in range(300):  # atau len(os.listdir(audio_dir)) jika fleksibel
    filename = f"{i}_kecapi.wav"
    path = os.path.join(audio_dir, filename)
    
    if os.path.exists(path):
        mfcc = extract_mfcc(path, sr=sample_rate, n_mfcc=n_mfcc)
        features.append([filename] + mfcc.tolist())
    else:
        print(f"File tidak ditemukan: {filename}")

# --- Simpan ke CSV ---
columns = ["filename"] + [f"mfcc_{i+1}" for i in range(n_mfcc)]
df = pd.DataFrame(features, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Selesai! Fitur MFCC disimpan di: {output_csv}")


import os
import librosa
import numpy as np
import pandas as pd

# --- Konfigurasi ---
audio_dir = "C:\\Users\\didi\\proposal skripsi\\Sitar5Detik"  # Ganti dengan path folder kamu
sample_rate = 16000
n_mfcc = 13  # Jumlah koefisien MFCC yang diambil
output_csv = "sitar_mfcc_features.csv"  # File hasil

# --- Ekstraksi MFCC ---
def extract_mfcc(file_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)  # Rata-rata per koefisien
    return mfcc_mean

# --- Loop untuk semua file ---
features = []
for i in range(300):  # atau len(os.listdir(audio_dir)) jika fleksibel
    filename = f"{i}_sitar.wav"
    path = os.path.join(audio_dir, filename)
    
    if os.path.exists(path):
        mfcc = extract_mfcc(path, sr=sample_rate, n_mfcc=n_mfcc)
        features.append([filename] + mfcc.tolist())
    else:
        print(f"File tidak ditemukan: {filename}")

# --- Simpan ke CSV ---
columns = ["filename"] + [f"mfcc_{i+1}" for i in range(n_mfcc)]
df = pd.DataFrame(features, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Selesai! Fitur MFCC disimpan di: {output_csv}")


import os
import librosa
import numpy as np
import pandas as pd

# --- Konfigurasi ---
audio_dir = "C:\\Users\\didi\\proposal skripsi\\ukulele5Detik"  # Ganti dengan path folder kamu
sample_rate = 16000
n_mfcc = 13  # Jumlah koefisien MFCC yang diambil
output_csv = "ukulele_mfcc_features.csv"  # File hasil

# --- Ekstraksi MFCC ---
def extract_mfcc(file_path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)  # Rata-rata per koefisien
    return mfcc_mean

# --- Loop untuk semua file ---
features = []
for i in range(300):  # atau len(os.listdir(audio_dir)) jika fleksibel
    filename = f"{i}_ukulele.wav"
    path = os.path.join(audio_dir, filename)
    
    if os.path.exists(path):
        mfcc = extract_mfcc(path, sr=sample_rate, n_mfcc=n_mfcc)
        features.append([filename] + mfcc.tolist())
    else:
        print(f"File tidak ditemukan: {filename}")

# --- Simpan ke CSV ---
columns = ["filename"] + [f"mfcc_{i+1}" for i in range(n_mfcc)]
df = pd.DataFrame(features, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Selesai! Fitur MFCC disimpan di: {output_csv}")

Selesai! Fitur MFCC disimpan di: kecapi_mfcc_features.csv
Selesai! Fitur MFCC disimpan di: sitar_mfcc_features.csv
Selesai! Fitur MFCC disimpan di: ukulele_mfcc_features.csv


In [54]:
import librosa
import numpy as np
import os

def lpc_to_lpcc(lpc_coeffs, order):
    lpcc = np.zeros(order)
    lpcc[0] = -np.log(lpc_coeffs[0])  # energy term
    for n in range(1, order):
        acc = 0
        for k in range(1, n):
            acc += (k / n) * lpcc[k] * lpc_coeffs[n - k]
        lpcc[n] = lpc_coeffs[n] + acc
    return lpcc

def extract_lpcc_from_file(file_path, lpcc_order=13, lpc_order=12):
    y, sr = librosa.load(file_path, sr=16000)
    lpc_coeffs = librosa.lpc(y, order=lpc_order)
    lpcc = lpc_to_lpcc(lpc_coeffs, lpcc_order)
    return lpcc

# Folder berisi file 0_gitar.wav, 1_kecapi.wav, dll
input_folder = "C:\\Users\\didi\\proposal skripsi\\GitarKlasik5Detik"
output_folder = "gitar_lpcc"
os.makedirs(output_folder, exist_ok=True)

for filename in sorted(os.listdir(input_folder)):
    if filename.endswith(".wav"):
        file_path = os.path.join(input_folder, filename)
        lpcc = extract_lpcc_from_file(file_path)
        out_name = os.path.splitext(filename)[0] + "_lpcc.npy"
        np.save(os.path.join(output_folder, out_name), lpcc)

print("LPCC extraction selesai untuk semua file.")

LPCC extraction selesai untuk semua file.


In [56]:
import librosa
import numpy as np
import os
import csv

def lpc_to_lpcc(lpc_coeffs, order):
    lpcc = np.zeros(order)
    lpcc[0] = -np.log(lpc_coeffs[0])  # energy term
    for n in range(1, order):
        acc = 0
        for k in range(1, n):
            acc += (k / n) * lpcc[k] * lpc_coeffs[n - k]
        lpcc[n] = lpc_coeffs[n] + acc
    return lpcc

def extract_lpcc_from_file(file_path, lpcc_order=13, lpc_order=12):
    y, sr = librosa.load(file_path, sr=16000)
    lpc_coeffs = librosa.lpc(y, order=lpc_order)
    lpcc = lpc_to_lpcc(lpc_coeffs, lpcc_order)
    return lpcc

# Folder input dan output CSV
input_folder = "C:\\Users\\didi\\proposal skripsi\\GitarKlasik5Detik"
csv_output_path = "gitar_lpcc_output.csv"

# Buka CSV dan tulis header + data
with open(csv_output_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    header = ["filename"] + [f"lpcc_{i}" for i in range(13)]
    writer.writerow(header)

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            lpcc = extract_lpcc_from_file(file_path)
            writer.writerow([filename] + lpcc.tolist())

print(f"LPCC disimpan ke dalam CSV: {csv_output_path}")

LPCC disimpan ke dalam CSV: gitar_lpcc_output.csv


In [3]:
import librosa
import numpy as np
import os
import csv

def lpc_to_lpcc(lpc_coeffs, order):
    lpcc = np.zeros(order)
    lpcc[0] = -np.log(lpc_coeffs[0])  # energy term
    for n in range(1, order):
        acc = 0
        for k in range(1, n):
            acc += (k / n) * lpcc[k] * lpc_coeffs[n - k]
        lpcc[n] = lpc_coeffs[n] + acc
    return lpcc

def extract_lpcc_from_file(file_path, lpcc_order=13, lpc_order=12):
    y, sr = librosa.load(file_path, sr=16000)
    lpc_coeffs = librosa.lpc(y, order=lpc_order)
    lpcc = lpc_to_lpcc(lpc_coeffs, lpcc_order)
    return lpcc

# Folder input dan output CSV
input_folder = "C:\\Users\\didi\\proposal skripsi\\harpa5Detik"
csv_output_path = "harpa_lpcc_output.csv"

# Buka CSV dan tulis header + data
with open(csv_output_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    header = ["filename"] + [f"lpcc_{i}" for i in range(13)]
    writer.writerow(header)

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            lpcc = extract_lpcc_from_file(file_path)
            writer.writerow([filename] + lpcc.tolist())

print(f"LPCC disimpan ke dalam CSV: {csv_output_path}")

# Folder input dan output CSV
input_folder = "C:\\Users\\didi\\proposal skripsi\\kecapi5Detik"
csv_output_path = "kecapi_lpcc_output.csv"

# Buka CSV dan tulis header + data
with open(csv_output_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    header = ["filename"] + [f"lpcc_{i}" for i in range(13)]
    writer.writerow(header)

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            lpcc = extract_lpcc_from_file(file_path)
            writer.writerow([filename] + lpcc.tolist())

print(f"LPCC disimpan ke dalam CSV: {csv_output_path}")

# Folder input dan output CSV
input_folder = "C:\\Users\\didi\\proposal skripsi\\Sitar5Detik"
csv_output_path = "sitar_lpcc_output.csv"

# Buka CSV dan tulis header + data
with open(csv_output_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    header = ["filename"] + [f"lpcc_{i}" for i in range(13)]
    writer.writerow(header)

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            lpcc = extract_lpcc_from_file(file_path)
            writer.writerow([filename] + lpcc.tolist())

print(f"LPCC disimpan ke dalam CSV: {csv_output_path}")

# Folder input dan output CSV
input_folder = "C:\\Users\\didi\\proposal skripsi\\ukulele5Detik"
csv_output_path = "ukulele_lpcc_output.csv"

# Buka CSV dan tulis header + data
with open(csv_output_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    header = ["filename"] + [f"lpcc_{i}" for i in range(13)]
    writer.writerow(header)

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            lpcc = extract_lpcc_from_file(file_path)
            writer.writerow([filename] + lpcc.tolist())

print(f"LPCC disimpan ke dalam CSV: {csv_output_path}")

LPCC disimpan ke dalam CSV: harpa_lpcc_output.csv
LPCC disimpan ke dalam CSV: kecapi_lpcc_output.csv
LPCC disimpan ke dalam CSV: sitar_lpcc_output.csv
LPCC disimpan ke dalam CSV: ukulele_lpcc_output.csv
